In [1]:
import torch
import torch.nn as nn

class RBFNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, sigma):
        super(RBFNet, self).__init__()
        self.hidden_dim = hidden_dim
        self.sigma = sigma
        self.centers = nn.Parameter(torch.randn(hidden_dim, input_dim))
        self.linear = nn.Linear(hidden_dim, output_dim)

    def radial_basis(self, x):
        # Compute the RBF activations
        dist = torch.cdist(x, self.centers)
        phi = torch.exp(-self.sigma * dist)
        return phi

    def forward(self, x):
        # Compute the forward pass of the network
        phi = self.radial_basis(x)
        output = self.linear(phi)
        return output


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and preprocess the data
data = pd.read_csv("D:\Datasets\Heart.csv")
data["ChestPain"] = data["ChestPain"].astype('category')
data.dtypes
data["ChestPain"]= data["ChestPain"].cat.codes
data.head()

data["Thal"] = data["Thal"].astype('category')
data.dtypes
data["Thal"]= data["Thal"].cat.codes
data.head()

data["AHD"] = data["AHD"].astype('category')
data.dtypes
data["AHD"]= data["AHD"].cat.codes
data.head()


X = data.iloc[:,5:9].values
y = data.iloc[:, -1].values
scaler = StandardScaler()
X = scaler.fit_transform(X)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.16)


# Train the network
num_epochs = 100
batch_size = 32
input_dim=X_train.shape[1]
hidden_dim=5 
output_dim=2
sigma=1.0
model = RBFNet(input_dim,hidden_dim,output_dim,sigma)
# Define the loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters())

for epoch in range(num_epochs):
    for i in range(0, X_train.shape[1], batch_size):
        # Forward pass
        inputs = torch.tensor(X_train[i:i+batch_size], dtype=torch.float32)
        labels = torch.tensor(y_train[i:i+batch_size], dtype=torch.long)
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    # Compute the accuracy on the test set
    with torch.no_grad():
        inputs = torch.tensor(X_test, dtype=torch.float32)
        labels = torch.tensor(y_test, dtype=torch.long)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == labels).sum()


In [3]:
print(accuracy)

tensor(25)


In [4]:
print(predicted)

tensor([0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0,
        0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0,
        0])


In [5]:
print(labels)

tensor([0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 1,
        0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0,
        0])


In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Load and preprocess the data
data = pd.read_csv("D:\Datasets\Heart.csv")
data["ChestPain"] = data["ChestPain"].astype('category')
data.dtypes
data["ChestPain"]= data["ChestPain"].cat.codes
data["Thal"] = data["Thal"].astype('category')
data.dtypes
data["Thal"]= data["Thal"].cat.codes
data["AHD"] = data["AHD"].astype('category')
data.dtypes
data["AHD"]= data["AHD"].cat.codes
data.head(10)

,Unnamed: 0,Age,Sex,ChestPain,RestBP,Chol,Fbs,RestECG,MaxHR,ExAng,Oldpeak,Slope,Ca,Thal,AHD
0,1,63,1,3,145,233,1,2,150,0,2.3,3,0.0,0,0
1,2,67,1,0,160,286,0,2,108,1,1.5,2,3.0,1,1
2,3,67,1,0,120,229,0,2,129,1,2.6,2,2.0,2,1
3,4,37,1,1,130,250,0,0,187,0,3.5,3,0.0,1,0
4,5,41,0,2,130,204,0,2,172,0,1.4,1,0.0,1,0
5,6,56,1,2,120,236,0,0,178,0,0.8,1,0.0,1,0
6,7,62,0,0,140,268,0,2,160,0,3.6,3,2.0,1,1
7,8,57,0,0,120,354,0,0,163,1,0.6,1,0.0,1,0
8,9,63,1,0,130,254,0,2,147,0,1.4,2,1.0,2,1
9,10,53,1,0,140,203,1,2,155,1,3.1,3,0.0,2,1
